# Gerekli Kütüphaneler

In [3]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from sklearn.datasets import make_blobs
from sklearn import metrics

seed = 42
np.random.seed(seed)
random.seed(seed)

# Veri Seti

In [4]:
X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=seed)

# DBSCAN

In [5]:
class DBSCAN:
    def __init__(self, eps=0.5, min_samples=5):
        # Komsuluk mesafesi (eps)
        self.eps = eps
        # Minimum komsu sayisi
        self.min_samples = min_samples
        # Etiketler (Gurultu icin -1 alir)
        self.labels = None

    def fit(self, X):
        n_samples = X.shape[0]
        # Baslangicta tum noktalar gurultu olarak kabul edilir (-1)
        self.labels = -np.ones(n_samples)
        cluster_id = 0

        for point_idx in range(n_samples):
            if self.labels[point_idx] != -1:
                continue

            # Komsulari bul
            neighbors = []
            for i in range(X.shape[0]):
                if np.linalg.norm(X[point_idx] - X[i]) <= self.eps:
                    neighbors.append(i)

            # Eger komsu sayisi min_samples'dan az ise, bu nokta gurultudur
            if len(neighbors) < self.min_samples:
                self.labels[point_idx] = -1
            else:
                # Yeni bir kume olustur ve genislet
                self.labels[point_idx] = cluster_id
                queue = deque(neighbors)
        
                while queue:
                    neighbor_idx = queue.popleft()
        
                    if self.labels[neighbor_idx] == -1:
                        # Gurultu noktalarini kumeye ekle
                        self.labels[neighbor_idx] = cluster_id
        
                    if self.labels[neighbor_idx] != -1:
                        continue
        
                    # Bu noktayi kumeye ekle
                    self.labels[neighbor_idx] = cluster_id
        
                    # Bu komsunun komsularini bul
                    new_neighbors  = []
                    for i in range(X.shape[0]):
                        if np.linalg.norm(X[point_idx] - X[i]) <= self.eps:
                            new_neighbors .append(i)
        
                    if len(new_neighbors) >= self.min_samples:
                        queue.extend(new_neighbors)

                cluster_id += 1

    def predict(self, X):
        return self.labels

# Eğitim

In [6]:
dbs = DBSCAN(eps=1.5, min_samples=2)
dbs.fit(X)

# Sonuçlar

In [7]:
cluster_centers = dbs.predict(X)

In [8]:
print(f"Homogeneity: {metrics.homogeneity_score(y, cluster_centers):.3f}")
print(f"Completeness: {metrics.completeness_score(y, cluster_centers):.3f}")
print(f"V-measure: {metrics.v_measure_score(y, cluster_centers):.3f}")
print(f"Adjusted Rand Index: {metrics.adjusted_rand_score(y, cluster_centers):.3f}")
print(f"Adjusted Mutual Information: {metrics.adjusted_mutual_info_score(y, cluster_centers):.3f}")
print(f"Silhouette Coefficient: {metrics.silhouette_score(X, cluster_centers):.3f}")

Homogeneity: 0.694
Completeness: 0.362
V-measure: 0.476
Adjusted Rand Index: 0.306
Adjusted Mutual Information: 0.431
Silhouette Coefficient: 0.214
